# Text Parser Testing document

This document is going to read the text file found in ../generated_data/Logs2020OCR_avepdf.com_horizontal.txt which was generated using the following website:


[avepdf.com](http://avepdf.com/)

In [1]:
import re # Regulra expresion library

In [2]:
#filename='../generated_data/Logs2020OCR_avepdf.com_horizontal.txt'
filename='../generated_data/Logs2019OCR_avepdf.com_horizontal.txt'

In [3]:
# read file in as a list of lines
lines = []
with open(filename) as f:
    lines = f.readlines()
    
for ind in range(len(lines)):
    lines[ind] = lines[ind].lstrip().rstrip()

In [4]:
#lines

In [5]:
# Parse cells using 20-1 regular expresisons
# Result is a list of list where each call is a list of lines for the call.
calls = []

call = []
for line in lines:
    if line == '':
        pass
    else:
        if re.match("[1-2][9,0]-[0-9]+\s", line[:10]):
            if len(call) > 0:
                calls.append(call)
                call = []
        call.append(line)
if len(call) > 0:
    call.append(line)

In [6]:
len(calls)

11619

In [7]:
calls[-1]

['19-16066                     2223           Initiated            - MOTOR   VEHICLE    STOP                                   CITATION     - WARNING',
 'Call      Taker:              PATROL  KALVIN    DZIEDZIAK',
 'Location/Address:                      COLD  SPRING   RD',
 'Unit:               30',
 'Arvd-22:23:36              Clrd-22:29:55',
 'Vehicle:                WHI   2013    FORD  EXPLOR     Reg:    PC  MA  8PB890      VIN:      1FM5KSF83DGB02476',
 'Operator:                 TATRO,   CAITLIN      R@    826   WALKER  ST   - CLARKSBURG,     MA  01247',
 'Race:     W    Sex:     F',
 'Refer      To   Citation:                  T2085797',
 'Owner:            TATRO,    JAMES   MICHAEL@    826    WALKER ST   - CLARKSBURG,      MA  01247-2885',
 'Race:     U     Sex:    M']

In [33]:
def parseHeader(line):
    print(line)
    index_past = 0
        
    callNumberMatch = re.search("[19,20]-\d+", line)
    if callNumberMatch is not None:
        callNumber = callNumberMatch.group()
        index_past = callNumberMatch.span()[1] + 1
    else:
        callNumber = None
        
    timeMatch = re.search("\d\d\d\d", line[index_past:])
    if timeMatch is not None:
        callTime = timeMatch.group()
        index_past += timeMatch.span()[1]+1
    else:
        callTime = None
        
    spaces = 0
    index = len(line)-1
    while spaces < 8:
        if line[index] == " ":
            spaces += 1
        else:
            spaces = 0
        index -= 1
    callReason = re.sub("\s+", " ", line[index_past:index])
    if callReason[0] == " ":
        callReason = callReason[1:]
    if callReason[-1] == " ":
        callReason = callReason[:-1]
    callAction = re.sub("\s+", " ", line[index+8:])
    if callAction[0] == " ":
        callAction = callAction[1:]
    if callAction[-1] == " ":
        callAction = callAction[:-1]
        
    return [callNumber, callTime, callReason, callAction]

def get_unit_times(unit_str):
    print(unit_str)
    times = re.sub(' +', ' ',unit_str).split(' ')
    tm_dict = {}
    for tm in times:
        if '-' in tm:
            vals = tm.split('-')
            tm_dict[vals[0]] = vals[-1]
    return tm_dict

def parse_call_list(call):
    '''Parse a call list into a dictionary'''
    
    if len(call) == 0:
        return
    
    my_call = {}
    
#     header = parseHeader(call[0])
#     my_call['callNumber'] = header[0]
#     my_call['callTime'] = header[1]
#     my_call['callReason'] = header[2]
#     my_call['callAction]']= header[3]
    my_call['header'] = call[0]
    
    

    
    ind = 0
    individual = ''
    while ind < len(call):
        line = call[ind]
        myline = line.split(':')
        if len(myline) == 2:
            tag = myline[0].rstrip().lstrip()
            tag = re.sub(' +', ' ',tag)
            value = myline[1].rstrip().lstrip()
            if tag == 'Narrative':
                if tag in my_call:
                    narrative = my_call['Narrative']
                else:
                    narrative = ''
                ind +=1
                while ind < len(call):
                    myline = call[ind].split(':')
                    if len(myline) == 1:
                        narrative += re.sub(' +', ' ',call[ind]) + " "
                    ind += 1
                my_call['Narrative'] = narrative
            else:
                if tag == "Unit":
                    ind += 1;
                    if ind < len(call):
                        unit_info = get_unit_times(call[ind])
                        if 'Units' in my_call:
                            my_call['Units'].append((value, unit_info))
                        else:
                            my_call['Units'] = [(value, unit_info)]
                else:
                    if tag == "Operator" or tag == "Owner":
                        individual=tag+"_"
                    my_call[tag] = value
        else:
            if len(myline) > 2:
                tags = re.findall('[\S]+:', line)

                for tag in reversed(tags):
                    start = line.rindex(tag)
                    value = line[start+len(tag):].rstrip().lstrip()

                    if len(value) > 0:
                        tag = tag[:-1]
                        my_call[individual+tag] = value
                        line = line[:line.rindex(tag)]
        ind +=1
    return(my_call)

In [34]:
#Parse an individual call
parse_call_list(calls[44])

Arvd-10:30:55          Clrd-10:31:49


{'header': '19-62                       1030         Initiated         - BUILDING     CHECK                                      BUILDING  CHECKED/SECURED',
 'Call Taker': 'PATROL TANIA  HERNANDEZ',
 'Location/Address': 'NORTH HOOSAC RD',
 'Units': [('36', {'Arvd': '10:30:55', 'Clrd': '10:31:49'})],
 'Narrative': 'checked 01/03/2019 0158 Checked S3 '}

In [35]:
#Parse all of the calls
call_dicts =[]
unit = ''
for call in calls:
    my_call = parse_call_list(call)
    call_dicts.append(my_call)
    print(my_call['header'])

1
Arvd-03:42:00          Clrd-03:42:17
19-1                          0341         Initiated         - PARKING   CHECK                                        SERVICES  RENDERED
Arvd-08:34:38          Clrd-08:35:33
19-4                         0834          Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
Arvd-08:43:07          Clrd-08:43:21
19-5                         0842         Initiated          - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
Arvd-08:46:51          Clrd-08:47:07
19-6                         0846         Initiated          - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
Arvd-08:47:37           Clrd-08:48:05
19-7                         0847         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
Arvd-08:59:22          Clrd-09:00:14
19-8                         0859         Initiated        

Arvd-17:55:09           Clrd-18:03:52
19-1068                   1755         Initiated          - MOTOR VEHICLE  STOP                              CITATION   - CIVIL
Arvd-18:20:26          Clrd-18:20:37
19-1069                   1819         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
Arvd-19:37:43          Clrd-19:39:58
19-1071                    1937         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
Arvd-20:02:58          Clrd-20:13:35
19-1072                   2002         Initiated          - MOTOR VEHICLE  STOP                              CITATION   - WARNING
19-1075                   2103         Phone   - BUILDING    CHECK                                               LOG ENTRY REQUEST
Disp-00:06:34          Enrt-00:06:41          Arvd-00:09:04           Clrd-00:14:10
19-1076                    0002        Phone   - SUSPICIOUS   MOTOR VEHICLE                    

19-2041                       0702           Phone     - WELL    BEING    CHECK                                                 LOG  ENTRY  REQUEST
19-2042                       0815           Other      - PUBLIC      RECORDS  REQUEST                                  MAILED
Arvd-08,22:42              Clrd-08:23:27
19-2043                       0822           Initiated            - BUILDING       CHECK                                            BUILDING     CHECKED/SECURED
Arvd-08:23:57          Clrd-08:24:18
19-2044                   0823         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
Arvd-08:26:22          Clrd-08:26:52
19-2045                   0826         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
Arvd-08:29:41          Clrd-08:30:06
19-2046                    0829         Initiated         - BUILDING    CHECK                                      BUILDING   CHEC

Arvd-10:10:06          Clrd-10:10:15
19-2990                    1010         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
Arvd-10:11:41          Clrd-10:12:10
19-2991                    1011         Initiated         - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
Arvd-10:13:09          Clrd-10:14:21
19-2992                   1012         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
Arvd-10:20:11          Clrd-10:21:10
19-2994                   1019         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
Arvd-10:20:28          Clrd-10:20:51
19-2995                   1020         Initiated         - BUILDING     CHECK                                     BUILDING   CHECKED/SECURED
Arvd-10:22:00          Clrd-10:22:13
19-2996                   1021         Initiated         - BUILDING     

Arvd-08:51:58          Clrd-08:52:04
19-3586                   0851         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
Arvd-08:55:25          Clrd-08:55:44
19-3587                   0854         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
Arvd-08:57:36          Clrd-08:57:48
19-3588                   0857         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
Arvd-08:57:58              Clrd-08:58:12
19-3589                        0857           Initiated            - BUILDING       CHECK                                            BUILDING     CHECKED/SECURED
Arvd-08:59:46              Clrd-09:00:02
19-3590                        0859           Initiated~              BUILDING     CHECK                                            BUILDING     CHECKED/SECURED
Disp-09:02:49              Enrt-09:03:08              Arvd-09

...

Disp-14:19:28              Enrt-14:19:41                                                   Clrd-14:21:51
Disp-14:21:45              Enrt-14:21:45              Arvd-14:23:44              Clrd-14:41:57
19-12842                      0718           Initiated           - TRAFFIC       CONTROL                                         SERVICES    RENDERED
Arvd-07:39:14              Clrd-07:39:24
19-12843                      0739          Initiated            - BUILDING       CHECK                                            BUILDING     CHECKED/SECURED
Arvd-07:44:17              Clrd-08:00:16
Disp-15:01:12              Enrt-15:01:54              Arvd-15:08:08              Clrd-15:19:11
19-12845                     0744           Initiated            - TRAFFIC       CONTROL                                         SERVICES   RENDERED
Arvd-08:13:20              Clrd-08:33:30
19-12846                     0813           Initiated            - TRAFFIC      CONTROL                                    

Arvd-21:10,57          Clrd-21:16:20
19-13465                 2110         Initiated         - MOTOR VEHICLE  STOP                              CITATION  - WARNING
Arvd-21:40:42         Clrd-22:31:19
19-13466                 2140         Initiated         - MOTOR VEHICLE  STOP                             CRIMINAL   COMPLAINT
Arvd-00:58:44              Clrd-01:24:02
19-13467                     0058           Initiated            - MOTOR   VEHICLE     STOP                                  CITATION     - WARNING
Arvd-01:06:06              Clrd-01:07:05
19-13468                     0106           Initiated            - BUILDING       CHECK                                            BUILDING     CHECKED/SECURED
Arvd-01:38:02              Clrd-01:40:37
19-13469                     0137           Initiated            - SUSPICIOUS      MOTOR VEHICLE                       SERVICES   RENDERED
Arvd-01:42:47              Clrd-01:43:26
19-13470                     0142           Initiated         

19-14619                     0725           Initiated            - MOTOR   VEHICLE    STOP                                   CITATION     - WARNING
Arvd-07:45:43          Clrd-08:03:21
Disp-14:41:03                                          Arvd-14:41:10          Clrd-15:13:36
19-14620                 0745         Initiated         - TRAFFIC    CONTROL                                   SERVICES RENDERED
Arvd-08:13:59          Clrd-08:36:51
Disp-14:56:00          Enrt-15:01:10           Arvd-15:03:00         Clrd-15:19:12
19-14621                 0813         Initiated         - TRAFFIC    CONTROL                                   SERVICES RENDERED
Arvd-08:53:08          Clrd-08:53:36
Diep-22:18:00                                                                           Clrd-22:21:54
19-14622                 0852         Initiated         - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
19-14623                 0856        Initiated          - BUILDING 

Arvd-12:05:16              Clrd-12:05:56
19-15289                     1204           Initiated            - BUILDING       CHECK                                            ALARM  FALSE
Arvd-12:15:10              Clrd-12:16:18
19-15290                     1214           Initiated            - BUILDING       CHECK                                            BUILDING     CHECKED/SECURED
Arvd-12:19:47              Clrd-12:24:19
19-15291                     1219           Initiated            - BUILDING       CHECK                                            BUILDING     CHECKED/SECURED
Disp-12:28:03              Enrt-12:28:29              Arvd-12:30:24              Clrd-12:37:11
19-15292                     1226           Phone    - TRAFFIC       COMPLAINT                                                SERVICES   RENDERED
19-15293                      1236          Phone     - SUSPICIOUS      ACTIVITY                                              LOG  ENTRY  REQUEST
Arvd-12:44:52             

19-15712                 1427         Initiated         - MOTOR VEHICLE  STOP                              CITATION  - WARNING
2312
19-15713                 1447         Initiated         - TRAFFIC    CONTROL                                   SERVICES RENDERED
Disp-15:01:21              Enrt-15:02:53              Arvd-15:08,36              Clrd-15:13:31
19-15714                     1457           Phone    - TRAFFIC       COMPLAINT                                               UNABLE   TO  LOCATE
Arvd-15:16:46              Clrd-15:21:06
19-15715                     1516           Initiated            - PARKING     CHECK                                               SERVICES   RENDERED
Arvd-15:18:12              Clrd-15:21:58
19-15716                     1517           Initiated            - PARKING     CHECK                                               SERVICES   RENDERED
Arvd-17:07:55              Clrd-17:10:31
19-15717                     1704           Initiated            - SERVE  

In [40]:
call_dicts[1]

{'header': '19-1                          0341         Initiated         - PARKING   CHECK                                        SERVICES  RENDERED',
 'Call Taker': 'PATROL CRAIG A  EICHHAMMER',
 'Location/Address': '[WIA  249]   NORTH ST',
 'Units': [('31', {'Arvd': '03:42:00', 'Clrd': '03:42:17'})]}

In [78]:
total_stops = 0
tot = 0
total_race = {}
norace = 0
my_stops = []
for call in call_dicts:
    if 'Call Taker' in call:
        if "craig" in call['Call Taker'].lower():
            tot += 1
            #print(call['header'])
            if "STOP" in call['header']:
                my_stops.append(call)
                total_stops += 1
                if 'Operator_Race' in call:
                    race = call['Operator_Race'].lower()
                    if race in total_race:
                        total_race[race] = total_race[race]+1
                    else:
                        total_race[race] = 1
                else:
                    norace +=1

In [79]:
tot

228

In [80]:
total_stops # by hammer

23

In [81]:
total_race

{'u': 14, 'w': 7}

In [77]:
my_stops

[{'header': '19-222                          2356           Initiated            - MOTOR   VEHICLE    STOP                                    CITATION     - WARNING',
  'Call Taker': 'PATROL   CRAIG   A  EICHHAMMER',
  'Location/Address': 'MAIN    ST',
  'Units': [('31', {'Arvd': '23:56:15', 'Clrd': '01/06/2019@'})],
  '00:01': '00',
  'VIN': '2T3BFREV7GW424961',
  'Reg': 'PC  MA  8NP499',
  'Vehicle': 'BLK   2016     RAV4',
  'Operator': 'WALL,    STEVEN  M@    6  MARLOW DR   - PERU,      MA  01235-9210',
  'Operator_Sex': 'M',
  'Operator_Race': 'u',
  'Refer To Citation': 'T0905670',
  'Owner': 'WALL,    STEVEN   M@   6   MARLOW DR  - PERU,      MA  01235-9210',
  'Owner_Sex': 'M',
  'Owner_Race': 'U',
  'For Date': '01/06/2019                       Sunday'},
 {'header': '19-759                     0056         Initiated         - MOTOR VEHICLE   STOP                             CITATION   - WARNING',
  'Call Taker': 'PATROL CRAIG A   EICHHAMMER',
  'Location/Address': 'COLD SPRING 

In [ ]:
# # Print out only the narratives

# for call in call_dicts:
#     if "Narrative" in call:
#         print(call['Narrative'])
#         print('\n')

In [ ]:
from ipywidgets import interact
# use interact decorator to decorate the function, so the function can receive the slide bar's value with parameter x.
@interact(x=(0, 100))
def double_number(x):
    print("The double of %d is %d." % (x, x*2))
    
# add another slide bar to this function with the interact decorator.    
@interact(y=(0, 10))
def square_number(y):
    print("The square of %d is %d." % (y, y**2))